In [17]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip uninstall gensim -y
!pip install --no-cache-dir --force-reinstall gensim


Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3


You can safely remove it manually.


In [19]:
import os
import concurrent.futures
import openai
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric, remove_stopwords
df = pd.read_csv("NEWS_YAHOO_stock_prediction.csv")
import time
from tqdm import tqdm


RuntimeError: Compiled extensions are unavailable. If you've installed from a package, ask the package maintainer to include compiled extensions. If you're building Gensim from source yourself, install Cython and a C compiler, and then run `python setup.py build_ext --inplace` to retry. 

In [20]:
df.drop(columns="Unnamed: 0",inplace=True)
df.set_index("Date")


,ticker,category,title,content,Open,High,Low,Close,Adj Close,Volume,label
Date,,,,,,,,,,,
2020-01-27,AAPL,opinion,Apple Set To Beat Q1 Earnings Estimates Tech ...,Technology giant Apple NASDAQ AAPL is set ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,Tech Daily Intel Results Netflix Surge Appl...,The top stories in this digest are Intel s N...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,7 Monster Stock Market Predictions For The Wee...,S P 500 SPY \nThis week will be packed with e...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,Apple Earnings Preview 5G Launch Expanding S...,Reports Q1 2020 results on Tuesday Jan 28 ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2020-01-27,AAPL,opinion,Buy Surging Apple Microsoft Stock Before Qua...,On today s episode of Full Court Finance here ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
...,...,...,...,...,...,...,...,...,...,...,...
2012-07-31,AAPL,opinion,What s Driving China s Real Estate Rally Par...,In the preceding posts I examined the first t...,21.543928,21.846430,21.525715,21.812857,18.624512,462327600,1
2012-07-31,AAPL,opinion,50 Stocks With The Best Dividend Growth In Jul...,Here is a current sheet of companies with fast...,21.543928,21.846430,21.525715,21.812857,18.624512,462327600,1
2012-07-23,AAPL,opinion,Summer Heat Scorches Europe And U S,Europe flares as summer heat continues Summer...,21.228571,21.639286,20.989643,21.565357,18.413187,487975600,1


In [21]:

# Load dataset
df = df.dropna(subset=["content"])

# Preprocess text (custom pipeline)
def preprocess_text(text):
    filters = [lambda x: x.lower(), strip_punctuation, strip_numeric, remove_stopwords]
    return preprocess_string(text, filters)

df["cleaned_text"] = df["content"].apply(preprocess_text)

# Create dictionary and corpus
dictionary = corpora.Dictionary(df["cleaned_text"])
corpus = [dictionary.doc2bow(text) for text in df["cleaned_text"]]

# Number of topics
num_topics = 5

# Train LDA model
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)

# Extract topic affinities
topic_affinities = []
for doc in corpus:
    topics = lda_model.get_document_topics(doc, minimum_probability=0)
    topic_affinities.append([prob for _, prob in topics])

# Convert to DataFrame
df_topics = pd.DataFrame(topic_affinities, columns=[f"Topic_{i}" for i in range(num_topics)])
df_final = pd.concat([df, df_topics], axis=1)

# Save and display
df_final.to_csv("news_with_topics_gensim.csv", index=False)




In [3]:
import pandas as pd
import concurrent.futures
from tqdm import tqdm
from openai import OpenAI

# Load API key from file
with open("api.txt", "r") as file:
    api_key = file.read().strip()

# Initialize OpenAI client with OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    default_headers={  # Set headers here, as OpenRouter recommends
        "HTTP-Referer": "https://your-site.com",  # Optional: Replace with your site URL
        "X-Title": "Your Site Name",  # Optional: Replace with your site name
    }
)

In [5]:
df = pd.read_csv("news_with_topics_gensim.csv")

## Version 1: Parallel computing

In [13]:
def extract_scores(reply):
    """
    Use regex to extract the first float following 'Sentiment:' and 'Importance:'.
    """
    sentiment_pattern = r"Sentiment:\s*([-+]?\d*\.?\d+)"
    importance_pattern = r"Importance:\s*([-+]?\d*\.?\d+)"
    
    sentiment_match = re.search(sentiment_pattern, reply)
    importance_match = re.search(importance_pattern, reply)
    
    if sentiment_match and importance_match:
        sentiment_score = float(sentiment_match.group(1))
        importance_score = float(importance_match.group(1))
        return sentiment_score, importance_score
    else:
        raise ValueError("Could not extract scores from reply: " + reply)

def get_scores(row):
    """
    Function to get sentiment and importance scores for a row of content and title.
    """
    print(row["title"], row["content"])
    try:
        # Prepare the prompt
        prompt = f"""
        Analyze the following article:
        Title: {row['title']}
        Content: {row['content']}
        
        Provide two scores:
        1. Sentiment score (scale -1 to 1): How positive or negative is the sentiment of the article?
        2. Importance score (scale 0 to 1): How important is this article's content for its context?
        
        Format your response as:
        Sentiment: <sentiment_score>
        Importance: <importance_score>
        """
        
        # Call the OpenRouter API using Qwen
        response = client.chat.completions.create(
            model="qwen/qwen-turbo",  # Correct model name
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,
            temperature=0.7
        )
        
        # Instead of checking with hasattr() and len(), use getattr to avoid calling len() on None
        choices = getattr(response, "choices", None)
        if not choices:
            raise ValueError("No valid choices in API response.")
        
        # Get the reply text safely
        reply_content = getattr(choices[0].message, "content", None)
        if not reply_content:
            raise ValueError("No reply content found in API response.")
        reply_text = reply_content.strip()
        
        # Use regex to extract the scores
        sentiment_score, importance_score = extract_scores(reply_text)
        return sentiment_score, importance_score

    except Exception as e:
        print(f"Error processing row (title: {row.get('title', 'N/A')}): {e}")
        return None, None

def process_dataset(df):
    """
    Processes the dataset sequentially, appending sentiment and importance scores.
    """
    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
        results.append(get_scores(row))
    
    # Unpack the results into two new columns
    df['sentiment_score'], df['importance_score'] = zip(*results)
    return df

# Load your dataset (e.g., from a CSV file)
# df = pd.read_csv("your_dataset.csv")
# Here, we assume 'df' is already defined.

# For testing, sample 5 rows from the DataFrame
d = df.sample(20)

# Process the dataset sequentially
d = process_dataset(d)

print("Processing complete. Scores added to dataset.")

Processing:   0%|          | 0/20 [00:00<?, ?it/s]

Wedbush bullish despite  Apple s darkest day  Wedbush analyst Daniel Ives lowers his Apple  AAPL  9 8   target from  275 to  200 and calls the Q1 warning  Apple s darkest day  Ives   Apple s decisions over the coming months around pricing  future hardware redesigns  timing of 5G smartphones  and driving its core services business will have implications for years to come as Cupertino now faces the biggest fear among bulls which is an installed base  we estimate roughly 750 million active iPhones worldwide  that could stall out and not grow over the coming years and in a nightmare scenario decline  The analyst maintains an Outperform rating and says he remains bullish on the Apple story despite yesterday s  black eye  Source  Research note Now read 


Processing:   5%|▌         | 1/20 [00:02<00:44,  2.37s/it]

Will Advanced Micro Devices  AMD  Reach A New 52 Week High  Shares of Advanced Micro Devices   NASDAQ AMD   gained more than 3  in morning trading Wednesday  extending the run of positive momentum that has seen the trendy chipmaker gain more than 16  over the past four weeks 
AMD s recent run comes as analysts and investors continue to get more bullish on the company and its products  For example  earlier this week Baird analyst Tristan Gerra suggested that AMD could be positioned to gain market share in the data center space which is dominated by Intel   NASDAQ INTC   thanks to its recently launched EPYC architecture 
And that s not the only place where AMD is gaining market share  According to a recent  from PC benchmarking site PassMark  tests involving AMD central processing units accounted for almost 26  of all x86 compatible tests submitted in the most recent quarter  up from 21  in the prior quarter and 18  last year 
These results could indicate that AMD s Ryzen chips are helpi

Processing:  10%|█         | 2/20 [00:04<00:42,  2.38s/it]

...

Market Update   26 07 2016 Currencies
EUR USD   moved up after the better than expected data out of Germany  but was held back by the resistance around the 1 10 level  We can see there is another attempt to break through this resistance level at the moment 
USD JPY   is seeing another large drop as there are reports that the stimulus that the Japanese government is planning to announce will fall short on expectation  as it will be less than what was reported last week  
GBP USD    is seeing less and less volatility and appears to be trading in a range between the 1 305 and 1 3 level at the moment  However  we can expect this to change soon as we will be waiting for the GDP data out of the UK tomorrow and the FOMC statement  It remains to be seen how much upside there is at the moment  as we are expecting a rate cut and likely further measures next month by the BOE  The likelihood for this has significantly increased as only last week MPC Waele said that it was not certain he would supp

Processing:  95%|█████████▌| 19/20 [00:41<00:01,  1.73s/it]

Kashmir apple trade picks up again under shadow of militant attacks By Fayaz Bukhari SRINAGAR  Reuters    More than 10 000 trucks laden with apples left Indian ruled Kashmir this week in a sign that the territory is returning to normal  according to officials and government data  after India imposed a clampdown on the region in August  Involving some 3 5 million people  apples are the core of Indian Kashmir s economy  which went into a tailspin after phone and internet links were suspended and hundreds of people detained to prevent anti India protests from erupting in the streets  Thousands of trucks were lined up on the main highway connecting Kashmir  a Muslim majority territory claimed in whole by India and Pakistan and ruled in part by both  to the rest of India on Thursday  signaling a recovery in the trade half way through the harvest  Beginning Monday  more than 12 000 trucks left Kashmir  bringing the total amount of apples shipped out of the region this year to nearly 0 7 mill

Processing: 100%|██████████| 20/20 [00:42<00:00,  2.13s/it]

Error processing row (title: Kashmir apple trade picks up again under shadow of militant attacks): No valid choices in API response.
Processing complete. Scores added to dataset.


## Version 3: Sequential and sequential add to csv


In [14]:
def extract_scores(reply):
    """
    Use regex to extract the first float following 'Sentiment:' and 'Importance:'.
    """
    sentiment_pattern = r"Sentiment:\s*([-+]?\d*\.?\d+)"
    importance_pattern = r"Importance:\s*([-+]?\d*\.?\d+)"
    
    sentiment_match = re.search(sentiment_pattern, reply)
    importance_match = re.search(importance_pattern, reply)
    
    if sentiment_match and importance_match:
        sentiment_score = float(sentiment_match.group(1))
        importance_score = float(importance_match.group(1))
        return sentiment_score, importance_score
    else:
        raise ValueError("Could not extract scores from reply: " + reply)

def get_scores(row):
    """
    Function to get sentiment and importance scores for a row of content and title.
    """    
    try:
        # Prepare the prompt
        prompt = f"""
        Analyze the following article:
        Title: {row['title']}
        Content: {row['content']}
        
        Provide two scores:
        1. Sentiment score (scale -1 to 1): How positive or negative is the sentiment of the article?
        2. Importance score (scale 0 to 1): How important is this article's content for its context?
        
        Format your response as:
        Sentiment: <sentiment_score>
        Importance: <importance_score>
        """
    
        # Call the OpenRouter API using Qwen
        response = client.chat.completions.create(
            model="qwen/qwen-turbo",  # Correct model name
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,
            temperature=0.7
        )
    
        # Check for a valid response
        choices = getattr(response, "choices", None)
        if not choices:
            raise ValueError("No valid choices in API response.")
    
        reply_content = getattr(choices[0].message, "content", None)
        if not reply_content:
            raise ValueError("No reply content found in API response.")
    
        reply_text = reply_content.strip()
        # Use regex to extract the scores
        sentiment_score, importance_score = extract_scores(reply_text)
        return sentiment_score, importance_score
    
    except Exception as e:
        print(f"Error processing row (title: {row.get('title', 'N/A')}): {e}")
        return None, None



# File paths
input_csv = "news_with_topics_gensim.csv"            # Your original CSV file
output_csv = "scored_dataset.csv" # Updated CSV file with scores

# Load the CSV file
if os.path.exists(output_csv):
    df = pd.read_csv(output_csv, index_col=0)  # Load already processed file
else:
    df = pd.read_csv(input_csv, index_col=0)
    # Create new columns for the scores if they don't exist.
    df['sentiment_score'] = None
    df['importance_score'] = None

# Process the DataFrame sequentially row by row
for idx, row in df.iterrows():
    # Check if the row already has scores (i.e. not NaN)
    if pd.notna(row['sentiment_score']) and pd.notna(row['importance_score']):
        continue  # Skip already processed rows

    # Get scores for the current row
    sentiment_score, importance_score = get_scores(row)
    df.at[idx, 'sentiment_score'] = sentiment_score
    df.at[idx, 'importance_score'] = importance_score

    # Save progress after each row (or every few rows)
    df.to_csv(output_csv)
    print(f"Row {idx} processed and saved.")
    # Optional: sleep for a short period to avoid rate limiting
    time.sleep(1)

print("Processing complete. Scores added to dataset.")


Row AAPL processed and saved.


NameError: name 'time' is not defined

In [12]:
d

,ticker,Date,category,title,content,Open,High,Low,Close,Adj Close,Volume,label,cleaned_text,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,sentiment_score,importance_score
8492,AAPL,2017-10-24,news,Amazon could account for nearly half of US e c...,Amazon NASDAQ AMZN will account for nearly h...,39.072498,39.355000,39.049999,39.275002,37.274540,71028800,1,"['amazon', 'nasdaq', 'amzn', 'account', 'nearl...",0.002200,0.094458,0.619151,0.282019,0.002172,0.6,0.8
15664,AAPL,2013-05-15,opinion,Technology Ratio Face Off Google Vs Apple A...,This Daily chart of the GOOG AAPL ratio shows ...,15.684286,15.750000,15.084286,15.316071,13.358216,741613600,0,"['daily', 'chart', 'goog', 'aapl', 'ratio', 's...",0.365338,0.624655,0.003332,0.003356,0.003319,0.0,0.7
5376,AAPL,2018-08-03,news,Huawei sees smartphone shipments topping 200 m...,By Sijia Jiang HONG KONG Reuters China s H...,43.869999,44.279999,43.767502,44.235001,42.304600,95096400,1,"['sijia', 'jiang', 'hong', 'kong', 'reuters', ...",0.082669,0.000795,0.686639,0.229106,0.000792,0.8,0.9
15218,AAPL,2016-07-07,opinion,Alibaba Ventures Into Connected Cars Partners...,Alibaba Group Holding Limited NYSE BABA is...,23.924999,24.125000,23.905001,23.985001,22.248240,100558400,1,"['alibaba', 'group', 'holding', 'limited', 'ny...",0.001168,0.001174,0.908069,0.042223,0.047366,0.5,0.7
14784,AAPL,2014-10-03,news,Exclusive Facebook plots first steps into hea...,By Christina Farr and Alexei Oreskovic SAN FRA...,18.870001,19.047501,18.869286,18.961430,16.844200,178584000,1,"['christina', 'farr', 'alexei', 'oreskovic', '...",0.000554,0.030343,0.765529,0.000549,0.203025,0.0,0.8


## Version 2: sequential computing

In [ ]:

def extract_scores(reply):
    """
    Use regex to extract the first float following 'Sentiment:' and 'Importance:'.
    """
    sentiment_pattern = r"Sentiment:\s*([-+]?\d*\.?\d+)"
    importance_pattern = r"Importance:\s*([-+]?\d*\.?\d+)"
    
    sentiment_match = re.search(sentiment_pattern, reply)
    importance_match = re.search(importance_pattern, reply)
    
    if sentiment_match and importance_match:
        sentiment_score = float(sentiment_match.group(1))
        importance_score = float(importance_match.group(1))
        return sentiment_score, importance_score
    else:
        raise ValueError("Could not extract scores from reply: " + reply)

def get_scores(row):
    """
    Function to get sentiment and importance scores for a row of content and title.
    """
    print(row["title",row["content"]])
    try:
        # Prepare the prompt
        prompt = f"""
        Analyze the following article:
        Title: {row['title']}
        Content: {row['content']}
        
        Provide two scores:
        1. Sentiment score (scale -1 to 1): How positive or negative is the sentiment of the article?
        2. Importance score (scale 0 to 1): How important is this article's content for its context?
        
        Format your response as:
        Sentiment: <sentiment_score>
        Importance: <importance_score>
        """
        
        # Call the OpenRouter API using Qwen
        response = client.chat.completions.create(
            model="qwen/qwen-turbo",  # Correct model name
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,
            temperature=0.7
        )
        
        # Check if the response contains valid choices
        if not response or not hasattr(response, "choices") or len(response.choices) == 0:
            raise ValueError("No valid choices in API response.")
        
        reply = response.choices[0].message.content.strip()
        # Extract the scores using regex
        sentiment_score, importance_score = extract_scores(reply)
        return sentiment_score, importance_score

    except Exception as e:
        print(f"Error processing row (title: {row.get('title', 'N/A')}): {e}")
        return None, None

def process_dataset(df):
    """
    Processes the dataset sequentially, appending sentiment and importance scores.
    """
    results = []
    # Iterate over each row using a simple loop with tqdm for progress visualization
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
        results.append(get_scores(row))
    
    # Unpack the results into two new columns
    df['sentiment_score'], df['importance_score'] = zip(*results)
    return df

# Load your dataset (for example, from a CSV file)
# df = pd.read_csv("your_dataset.csv")
# For demonstration, we'll assume 'df' is already defined.

# Sample 5 rows for testing
d = df.sample(5)

# Process the dataset without parallel computing
d = process_dataset(d)

# Save or print the updated dataset
print("Processing complete. Scores added to dataset.")

NameError: name 'df' is not defined